# MC Stat Expected
Matt Kretchmar  
September 2022

This notebook contains code to analyze historical data in XC results in Ohio from 2017 through 2021 seasons.  The primary objective here is to tabulate the **ACTUAL** cell counts for each category in the table below.   

We organize the data into the following for each division separately:

| | Smallest 25% | Middle 25% | Biggest 25% |
|-|-|-|-|
| Districts | a | b | c |
| Regionals | d | e | f | 
| States | g | h | i | 


We draw from 30 different datasets corresponding to:
- Two genders: (boys and girls)
- Three divisions: (I, II, and III)
- Five years (2017-2021)


There is enough statistical similarity from year to year, so we were able to combine and average the data across the five years.  This generates **SIX** data sets corresponding to the two genders and three divisions (which are not similar enough to combine).  

This notebook produces the ACTUAL cell counts for these six different experiments/datasets.  




In [2]:
import pandas as pd
import sqlalchemy as sa
import numpy as np
import math
import random


### Create Database Connection
Here we connect to our database engine.  For anyone wanting to reproduce this portion of the work, you will need to download the database and install it on your own mysql server.   Then you will need to alter the IP address name to connect to your own engine.

Put your login credentials in the `creds.txt` file.  

In [3]:
# read credentials
fileHD = open('creds.txt')
creds = fileHD.read()
fileHD.close()
userid,password = creds.split()

# make connection
pattern = "{}://{}:{}@{}/{}"
protocol = 'mysql+mysqlconnector'
server = "jakku.cs.denison.edu"
database = 'OH_XC'
cstring = pattern.format(protocol,userid,password,server,database)
#print(cstring)

# connect to DB engine
engine = sa.create_engine(cstring)
connection = engine.connect()

### Main Computation
The following cell contains the main computation for this notebook.  
We iterate over 30 experiments
- Divisions: 1, 2, 3
- Genders: B, G
- Years: 2017-2021

For each experiment, we query the Database for 
1. District Race List
2. Regional Race List
3. State Race List

We sort the lists by school population and then use that sorted data to select
- The smallest 25% for the SMALL group
- The middle 50% for the MIDDLE group
- The largest 25% for the BIG group

We then tabulate the cell counts for actual schools in each category

In [25]:
for division in [1,2,3]:
    for gender in ['B','G']:
        grid = [[0,0,0] for i in range(3)]
        for year in ['2017','2018','2019','2020','2021']:

            # Process the DISTRICT race list
            q1 = "SELECT S.BLDG_IRN, S.NAME, S.POPULATION, X.GENDER, X.DIVISION, X.YEAR, X.TYPE "
            q2 = "FROM XC_Results as X INNER JOIN School as S USING(BLDG_IRN) "
            q3 = "WHERE X.YEAR={0} AND X.DIVISION={1} AND X.GENDER='{2}' ".format(year,division,gender)
            q4 = "AND X.TYPE='District';"

            query = q1+q2+q3+q4
            #print('\n\n*****\n',query,'\n')

            resultproxy = connection.execute(query)
            df = pd.DataFrame(resultproxy.fetchall(),columns=resultproxy.keys())
            df = df.sort_values(by=['POPULATION'])
            #print(df)

            n = len(df)
            cut1 = int(n / 4)
            cut2 = n-cut1

            # Group District results into small, middle, big
            df_small = df.iloc[:cut1]
            df_big = df.iloc[cut2:]
            df_middle = df.iloc[cut1:cut2]

            #print('len of small',len(df_small))
            #print('len of middle',len(df_middle))
            #print('len of big',len(df_big))

            small_list = df_small.BLDG_IRN.tolist()
            middle_list = df_middle.BLDG_IRN.tolist()
            big_list = df_big.BLDG_IRN.tolist()
            
            # select DISTRICT results where schools are not in the Regional results
            q1 = "SELECT S.BLDG_IRN, S.NAME, S.POPULATION, X.GENDER, X.DIVISION, X.YEAR, X.TYPE "
            q2 = "FROM XC_Results as X INNER JOIN School as S USING(BLDG_IRN)"
            q3 = "WHERE X.YEAR={0} AND X.DIVISION={1} AND X.GENDER='{2}' ".format(year,division,gender)
            q4 = "AND X.TYPE='District' AND X.BLDG_IRN NOT IN "
            q5 = "(SELECT BLDG_IRN FROM XC_Results WHERE "
            q6 = "YEAR={0} AND GENDER='{1}' AND DIVISION={2} and TYPE='Regional') ".format(year,gender,division)
            q7 = "ORDER BY S.POPULATION ASC;"


            query = q1+q2+q3+q4+q5+q6+q7
            #print('\n\n*****\n',query,'\n')


            resultproxy = connection.execute(query)
            df2 = pd.DataFrame(resultproxy.fetchall(),columns=resultproxy.keys())
            #print(df2)

            #print('length df2',len(df2))
            district_list = df2.BLDG_IRN.tolist()
            for ID in district_list:
                if ID in small_list:
                    grid[0][0] += 1
                elif ID in middle_list:
                    grid[0][1] += 1
                elif ID in big_list:
                    grid[0][2] += 1

            # select regional results where schools are not in the State Results
            q1 = "SELECT S.BLDG_IRN, S.NAME, S.POPULATION, X.GENDER, X.DIVISION, X.YEAR, X.TYPE "
            q2 = "FROM XC_Results as X INNER JOIN School as S USING(BLDG_IRN)"
            q3 = "WHERE X.YEAR={0} AND X.DIVISION={1} AND X.GENDER='{2}' ".format(year,division,gender)
            q4 = "AND X.TYPE='Regional' AND X.BLDG_IRN NOT IN "
            q5 = "(SELECT BLDG_IRN FROM XC_Results WHERE "
            q6 = "YEAR={0} AND GENDER='{1}' AND DIVISION={2} and TYPE='State') ".format(year,gender,division)
            q7 = "ORDER BY S.POPULATION ASC;"


            query = q1+q2+q3+q4+q5+q6+q7
                    
            resultproxy = connection.execute(query)
            df2 = pd.DataFrame(resultproxy.fetchall(),columns=resultproxy.keys())
            #print(df2)

            #print('length df2',len(df2))
            regional_list = df2.BLDG_IRN.tolist()
            for ID in regional_list:
                if ID in small_list:
                    grid[1][0] += 1
                elif ID in middle_list:
                    grid[1][1] += 1
                elif ID in big_list:
                    grid[1][2] += 1
               
            # select the state results
            q1 = "SELECT S.BLDG_IRN, S.NAME, S.POPULATION, X.GENDER, X.DIVISION, X.YEAR, X.TYPE "
            q2 = "FROM XC_Results as X INNER JOIN School as S USING(BLDG_IRN)"
            q3 = "WHERE X.YEAR={0} AND X.DIVISION={1} AND X.GENDER='{2}' ".format(year,division,gender)
            q4 = "AND X.TYPE='State';"


            query = q1+q2+q3+q4
                    
            resultproxy = connection.execute(query)
            df2 = pd.DataFrame(resultproxy.fetchall(),columns=resultproxy.keys())
            #print(df2)

            #print('length df2',len(df2))
            state_list = df2.BLDG_IRN.tolist()
            for ID in state_list:
                if ID in small_list:
                    grid[2][0] += 1
                elif ID in middle_list:
                    grid[2][1] += 1
                elif ID in big_list:
                    grid[2][2] += 1
                    
        # display final cell counts       
        print('\n\n\nData as Counts Div {0}   Gender {1}'.format(division,gender))
        print('{0:5.1f}  {1:5.1f}  {2:5.1f}'.format(grid[0][0],grid[0][1],grid[0][2]))
        print('{0:5.1f}  {1:5.1f}  {2:5.1f}'.format(grid[1][0],grid[1][1],grid[1][2]))
        print('{0:5.1f}  {1:5.1f}  {2:5.1f}'.format(grid[2][0],grid[2][1],grid[2][2]))
            
            





Data as Counts Div 1   Gender B
184.0  361.0  111.0
 47.0   96.0   80.0
 12.0   35.0   52.0



Data as Counts Div 1   Gender G
147.0  268.0   91.0
 47.0  116.0   63.0
 13.0   34.0   54.0



Data as Counts Div 2   Gender B
182.0  334.0  120.0
 48.0  118.0   72.0
 11.0   40.0   49.0



Data as Counts Div 2   Gender G
155.0  272.0   88.0
 46.0  122.0   68.0
 10.0   36.0   55.0



Data as Counts Div 3   Gender B
188.0  297.0  138.0
 33.0  132.0   67.0
 16.0   53.0   32.0



Data as Counts Div 3   Gender G
165.0  229.0  111.0
 31.0  119.0   62.0
  7.0   64.0   30.0
